In [15]:
import csv
import math
import random

In [16]:
def loadCsv(filename):
    lines= csv.reader(open(r''+filename))
    dataset= list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

In [17]:
def splitDataset(dataset, splitRatio):
    trainSize= int(len(dataset) * splitRatio)
    trainSet= []
    copy= list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

In [18]:
# saprate train class instances bt class value so that we can calculate statistics of every attribute
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if(vector[-1] not in separated):
            separated[vector[-1]]=[]
        separated[vector[-1]].append(vector)
    return separated

In [19]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

In [20]:
def stddev(numbers):
    avg=mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [21]:
#summarize by class
def summarize(dataset):
    summaries = [(mean(attribute),stddev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [22]:
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue,instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [23]:
def calculateProbability(x, mean, stddev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stddev,2))))
    return (1/(math.sqrt(2*math.pi)*stddev))*exponent

In [24]:
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean,stddev = classSummaries[i]
            x= inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stddev)
    return probabilities        

In [25]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [26]:
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions
    

In [27]:
def getAccuract(testSet, predictions):
    correct = 0 
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet)))*100.0

In [28]:
def main():
    filename = 'pima-indians-diabetes.csv'
    splitRatio = 0.67
    dataset = loadCsv(filename)
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('Split {0} rows into train = {1} and test = {2} rows'.format(len(dataset),len(trainingSet),len(testSet)))
    #prepare model
    summaries = summarizeByClass(trainingSet)
    #test model
    predictions = getPredictions(summaries, testSet)
    accuracy = getAccuract(testSet, predictions)
    print('Accuract: {0}%'.format(accuracy))
    
main()   

Split 768 rows into train = 514 and test = 254 rows
Accuract: 73.62204724409449%
